# Synthetix Perps V2 Funding

This project provides a Python based sample implementation of the new funding model described in [SIP-279](https://sips.synthetix.io/sips/sip-279/) for Perps V2. The new funding model aims to optimise for a perfectly balanced skew. To recap the basics,

1. A perps market is a Synth/sUSD pair e.g. sETH/sUSD
1. A market skew is present when longs and shorts don't balance out e.g. 1M short, 2M long has a 1M long skew
1. When a skew is present, the skewed direction pays the opposite defined by a 'funding rate' e.g. 1M longs pay shorts. This is meant to incentivise arbers to short on Synthetix and long on another exchange, reducing skew back to equilibrium

The current mainnet implementation of [SIP-80](https://sips.synthetix.io/sips/sip-80/) poses two problems:

1. There is no financial incentive to perform the arb as funding rates are calculated instantaneously. In the example above, a short to bring skew back to 0 will also result in a 0 funding rate. The arber makes no money
1. Funding rate volatility. A funding rate that fluctuates too frequently is not attractive for traders

The new funding model in SIP-279 solves this by introducing a floating funding rate of sorts. The basic idea is as follows:

1. As long as the market skew is not at equilibrium, increase the funding in the same direction. As some point, there will be enough market participants willing to bring it back. If there's free money on the table, and you had alpha to take advantage, why not?
1. If/when a market skew is at equilibrium stop changing the funding rate. This is the floating funding rate. It doesn't instantaneously drop to 0, keeping the arbers on the exchange to maintain skew. We can denote this as the fair market funding rate.
1. If the skew drifts past equilibrium into the other direction, the funding rate hence increases in the opposite direction.

# Funding Velocity

This defines how quickly funding rate increases as long as the skew is not at equilibrium. We define 2 static but configurable variables

- `fundingVelocityScale` - A value in USD to fractionalise the skew e.g. 1M / 10M = 0.1
- `maxFundingVelocity` - The maximum percentage in funding per day e.g. 2.5%

Note that funding rate increases relative to the skew, fractionalised by `fundingVelocityScale`.

In [2]:
FUNDING_VELOCITY_SCALE = 10_000_000 # 10M USD
MAX_FUNDING_VELOCITY = 0.025 # 2.5%
ASSET_PRICE = 1500 # 1500 USD


def calc_funding_rate_velocity(skew: float, skew_scale_usd: float, max_funding_pct: float, price: float) -> float:
    """Calculates the daily funding rate given current market conditions.

    :param float skew: Market skew denoted by size
    :param float skew_scale_usd: Maximum funding velocity scale denoted in USD
    :param float max_funding_pct: Maximum percentage increase in funding per day
    :param float price: Price of the underlying market Synth e.g. sETH

    """
    return (skew * price) / skew_scale_usd * max_funding_pct

# Examples
print(calc_funding_rate_velocity(-1000, FUNDING_VELOCITY_SCALE, MAX_FUNDING_VELOCITY, ASSET_PRICE))
print(calc_funding_rate_velocity(10_000, FUNDING_VELOCITY_SCALE, MAX_FUNDING_VELOCITY, ASSET_PRICE))
print(calc_funding_rate_velocity(FUNDING_VELOCITY_SCALE / ASSET_PRICE, FUNDING_VELOCITY_SCALE, MAX_FUNDING_VELOCITY, ASSET_PRICE)) # max rate velocity

-0.00375
0.037500000000000006
0.025


# Example Simulation

We can simulate this behaviour to see how the funding rate can float. Let's start by defining a simple subset of a `Position` dataclass to represent positions on-chain for the purposes of this simulation. We also define a `Market` dataclass to isolate 

In [26]:
from dataclasses import dataclass
from typing import List, Type, Optional, Union


SECONDS_IN_DAY: int = 86400

@dataclass
class Position:
    size: float


@dataclass
class Market:
    positions: List[Type[Position]]
    price: float = 0
    last_modified_at: int = 0

    # Cached values on every market modification to avoid computing against `positions` every time.
    size: float = 0
    skew: float = 0
    funding_rate: float = 0
    funding_accrued: float = 0

    # Static but configurable variables
    skew_scale_usd: float = 0
    max_funding_pct: float = 0

    def modify_position(self, t: int, size_usd: Optional[float]) -> None:
        # Calculate how much time has passed from the last interaction to now.
        #
        # 1. Funding accrued between t and t + 1 using the old funding rate
        # 2. Update funding after adding the position.

        # How much time has passed?
        t_delta = t - self.last_modified_at 

        # Update accrued funding.
        self.funding_accrued += (t_delta * (self.funding_rate / SECONDS_IN_DAY)) * self.size * self.price

        # Update funding rate.
        self.funding_rate += t_delta * (self.funding_rate_velocity() / SECONDS_IN_DAY)
        self.last_modified_at = t

        # No position was added/removed. We're just touching to update cache.
        if size_usd:
            size = size_usd / self.price
            position = Position(size)
            self.positions.append(position)
            self.size += abs(size)
            self.skew += size

    def funding_rate_velocity(self) -> float:
        return (self.skew * self.price) / self.skew_scale_usd * self.max_funding_pct


In [ ]:
import sys
!{sys.executable} -m pip install tabletext

In [28]:
from pprint import pprint
import tabletext


def ppint_sim_events(events: List[List[Union[int, float, str]]]) -> None:
    columns = ['ts (seconds)', 'size', 'skew', 'funding_rate', 'velocity', 'funding_accrued (total in usd)']
    print(tabletext.to_text([columns] + events))


def simulation_01() -> None:
    """Concrete example with pre-defined timeseries (with static price)."""
    price = 1000
    intervals = [
        {'t': 1000, 'size_usd': 1_000_000},
        {'t': 30_000, 'size_usd': 2_000_000},
        {'t': 50_000, 'size_usd': -3_000_000},
        {'t': 100_000, 'size_usd': None},
        {'t': 150_000, 'size_usd': -2_000_000},
        {'t': 200_000, 'size_usd': 2_000_000},
        {'t': 300_000, 'size_usd': None},
        {'t': 400_000, 'size_usd': None},
    ]

    # Initial market conditions. We start with a perfectly balanced skew with 2M in both directions.
    market = Market(
        positions=[],
        last_modified_at=0,
        price=price,
        skew_scale_usd=FUNDING_VELOCITY_SCALE,
        max_funding_pct=MAX_FUNDING_VELOCITY,
    )

    market.modify_position(0, -2_000_000)
    market.modify_position(0, 2_000_000)

    events = [
        [0, 0, 0, 0, market.funding_rate_velocity(), 0]
    ]
    for interval in intervals:
        market.modify_position(interval['t'], interval['size_usd'])
        events.append([
            market.last_modified_at,
            market.size,
            market.skew,
            market.funding_rate,
            market.funding_rate_velocity(),
            market.funding_accrued,
        ])
    return events

sim_01_res = simulation_01()
ppint_sim_events(sim_01_res)

┌──────────────┬─────────┬─────────┬────────────────────────┬───────────────────────┬────────────────────────────────┐
│ ts (seconds) │ size    │ skew    │ funding_rate           │ velocity              │ funding_accrued (total in usd) │
├──────────────┼─────────┼─────────┼────────────────────────┼───────────────────────┼────────────────────────────────┤
│            0 │       0 │       0 │                      0 │                   0.0 │                              0 │
├──────────────┼─────────┼─────────┼────────────────────────┼───────────────────────┼────────────────────────────────┤
│         1000 │  5000.0 │  1000.0 │                    0.0 │ 0.0025000000000000005 │                            0.0 │
├──────────────┼─────────┼─────────┼────────────────────────┼───────────────────────┼────────────────────────────────┤
│        30000 │  7000.0 │  3000.0 │  0.0008391203703703706 │                0.0075 │                            0.0 │
├──────────────┼─────────┼─────────┼────────────

## Sim 01 - Observations

- We start with -2M in shorts and long, hence a perfectly balanced market. The funding rate is 0% and velocity is 0%
- 1000 seconds pass and a trader opens a 1M long position resulting in a 1000 size skew
  - Note funding is calculated based prior to adding the new position. Hence we see a 0%
- A further 29,000 seconds pass and another 2M long is added and skews the market further
  - Notice the velocity increasing proportional to the skew (0.0025 in the previous epoch 0.0075 in the current)
  - Funding rate is calculated based on the time delta between t - (t - 1).
- 20,000 seconds pass and a 2M short position is added, perfectly balancing the skew
- 50,000 seconds pass with no changes to the skew
  - Notice the funding rate does not change. This is the fair market funding rate and keeps arbers incentivised to stay on the exchange
- 50,000 seconds pass and a 2M short is added, pushing the skew in the negative direction
  - We start to see a change in funding rate in the other direction.
- 50,000 seconds pass, a 2M long is added. We are perfectly balanced.
  - If we touch on this more frequently, perhaps 150k, 160k, 170k, 180k, 190k, 200k, we will see a more gradual shift in funding rate in the opposite direction.
- 100k - 200k seconds pass and no change in skew, velocity is at 0, and funding rate does not change